# Part 2: Cost of Living

In [1]:
import numpy as np
import pandas as pd
import re
import calendar

import seaborn as sns
import matplotlib.style as style

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

style.use('fivethirtyeight')

In [2]:
df = pd.read_csv('./data/all_jobs.csv')
salary_df = pd.read_csv('./data/only_salary_jobs.csv')

### About Cost of Living Indices

These indices are relative to New York City (NYC). Which means that for New York City, each index should be 100(%). If another city has, for example, rent index of 120, it means that on an average in that city rents are 20% more expensive than in New York City. If a city has rent index of 70, that means on average rent in that city is 30% less expensive than in New York City.

**Cost of Living Index (Excl. Rent)** is a relative indicator of consumer goods prices, including groceries, restaurants, transportation and utilities. Cost of Living Index does not include accommodation expenses such as rent or mortgage. If a city has a Cost of Living Index of 120, it means Numbeo has estimated it is 20% more expensive than New York (excluding rent).

**Rent Index** is an estimation of prices of renting apartments in the city compared to New York City. If Rent index is 80, Numbeo has estimated that price of rents in that city is on average 20% less than the price in New York.

**Groceries Index** is an estimation of grocery prices in the city compared to New York City. To calculate this section, Numbeo uses weights of items in the "Markets" section for each city.

**Restaurants Index** is a comparison of prices of meals and drinks in restaurants and bars compared to NYC.

**Cost of Living Plus Rent Index** is an estimation of consumer goods prices including rent comparing to New York City.

**Local Purchasing Power** shows relative purchasing power in buying goods and services in a given city for the average net salary in that city. If domestic purchasing power is 40, this means that the inhabitants of that city with an average salary can afford to buy on an average 60% less goods and services than New York City residents with an average salary.

*Source: [numbeo.com](https://www.numbeo.com/cost-of-living/cpi_explained.jsp)*
